<a href="https://colab.research.google.com/github/sensity-ai/dot/blob/update-colab-notebook/notebooks/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deepfake Offensive Toolkit

> **Disclaimer**: This notebook is primarily used for demo purposes on Google Colab.

**Note**: We recommend running this notebook on Google Colab with GPU enabled.

To enable GPU, do the following:

`Click "Runtime" tab > select "Change runtime type" option > set "Hardware accelerator" to "GPU"`

### Install Notebook Pre-requisites:

We install the following pre-requisities:
- `ffmpeg`
- `conda` (via [condacolab](https://github.com/conda-incubator/condacolab))

Note: The notebook session will restart after installing the pre-requisites.

**RUN THE BELOW CELL ONLY ONCE.**

**ONCE THE NOTEBOOK SESSION RESTARTS, SKIP THIS CELL MOVE TO "STEP 1" SECTION OF THIS NOTEBOOK**

In [ ]:
# install linux pre-requisites
!sudo apt install ffmpeg

# install miniconda3
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py310_23.3.1-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


## Step 1 - Clone Repository

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'dot'


In [ ]:
!git clone https://github.com/sensity-ai/dot.git $CODE_DIR


/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'dot'...
remote: Enumerating objects: 585, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 585 (delta 195), reused 136 (delta 90), pack-reused 284
Receiving objects: 100% (585/585), 16.65 MiB | 33.76 MiB/s, done.
Resolving deltas: 100% (258/258), done.


In [ ]:
os.chdir(f'./{CODE_DIR}')


## Step 2 - Setup Conda Environment

**ONCE THE INSTALLATION IS COMPLETE, RESTART THE NOTEBOOK AND MOVE TO "STEP 2" SECTION OF THIS NOTEBOOK**

In [ ]:
# update base conda environment: install python=3.8 + cudatoolkit=11.8
!conda install python=3.8 cudatoolkit=11.8

# install pip requirements
!pip install llvmlite==0.38.1 onnxruntime-gpu==1.9.0
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -r requirements.txt

# install dot
!pip install -e .


/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Solving environment: / - \ | / - \ | / - \ | / - \ WARNING conda.core.solve:_add_specs(641): pinned spec python=3.10 conflicts with explicit specs.  Overriding pinned spec.
| / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ WARNING conda.core.solve:_add_specs(641): pinned spec python=3.10 conflicts with explicit specs.  Overriding pinned spec.
| / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Obtaining file:///content/dot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for dot (pyproject.toml) ... done
  Created wheel for dot: filename=dot-1.1.0-0.editable-py3-none-any.whl size=8436 sha256=a16e3a7a5001d7d8fbe049af23b4b413659a14c72873d744f2e3658feebf351e
  Stored in directory: /tmp/pip-ephem-wheel-cache-u17udi_z/wheels/95/9f/46/753598c47299021ffeb65bed1b1893ac88324f4e169356e665
Successfully built dot


## Step 2 - Download Pretrained models

In [ ]:
%cd /content/dot

# download binaries
!gdown 1Qaf9hE62XSvgmxR43dfiwEPWWS_dXSCE

# unzip binaries
!unzip dot_model_checkpoints.zip

# clean-up
!rm -rf *.z*


/content/dot
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Downloading...
From: https://drive.google.com/uc?id=1Qaf9hE62XSvgmxR43dfiwEPWWS_dXSCE
To: /content/dot/dot_model_checkpoints.zip
100% 2.36G/2.36G [00:30<00:00, 76.4MB/s]
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Archive:  dot_model_checkpoints.zip
   creating: saved_models/faceswap_cv/
  inflating: saved_models/faceswap_cv/shape_predictor_68_face_landmarks.dat  
   creating: saved_models/fomm/
  inflating: saved_models/fomm/vox-adv-cpk.pth.tar  
   creating: saved_models/gpen/
   creating: saved_models/gpen/weights/
  inflating: saved_models/gpen/weights/GPEN-BFR-256.pth  
  inflating: saved_models/gpen/weights/GPEN-BFR-512.pth  
  inflating: saved_models/gpen/weights/GPEN-Inpainting-1024.pth  
  inflating: saved_models/gpen/

## Step 3: Run dot on image and video files instead of camera feed

### Using SimSwap on Images


In [ ]:
!dot \
-c ./configs/simswap.yaml \
--target "data/" \
--source "data/" \
--save_folder "image_simswap_output/" \
--use_image \
--use_gpu


/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 140MB/s]
0: Performed face swap ('data/Leonardo Dicaprio.jpg', 'data/eminem.jpg') saved to image_simswap_output/21105810.jpg
1: Performed face swap ('data/Joe Biden.jpg', 'data/schwarzenegger.png') saved to image_simswap_output/31108101.jpg
2: Performed face swap ('data/Tom Cruise.jpg', 'data/Leonardo Dicaprio.jpg') saved to image_simswap_output/1037907.jpg
3: Performed face swap ('data/schwarzenegger.png', 'data/Joe Biden.jpg') saved to image_simswap_output/672049.jpg
4: Performed face swap ('data/David Beckham.jpg', 'data/David Beckham.jpg') saved to image_simswap_output/248473.jpg
5: Performed face swap ('data/obama.jpg', 'data/Tom Cruise.jpg') saved to image_simswap_out

### Using SimSwap on Videos

In [ ]:
!dot \
-c ./configs/simswap.yaml \
--source "data/" \
--target "data/" \
--save_folder "video_simswap_output/" \
--limit 1 \
--use_video \
--use_gpu

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Total source images:  15
Total target videos:  1
Total number of face-swaps:  1
  0% 0/1 [00:00<?, ?it/s]OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Source: data/jobs.jpg 
Target: data/test_video.mp4 
Output: video_simswap_output/jobs_test_video.mp4 
FPS: 24 
Total frames: 1014

  0% 0/1014 [00:00<?, ?it/s]
  0% 1/1014 [00:00<03:23,  4.99it/s]
  0% 2/1014 [00:00<03:08,  5.36it/s]
  0% 3/1014 [00:00<03:06,  5.42it/s]
  0% 4/1014 [00:00<03:09,  5.33it/s]
  0% 5/1014 [00:00<03:22,  4.99it/s]
  1% 6/1014 [00:01<03:29,  4.80it/s]
  1% 7/1014 [00:01<03:30,  4.78it/s]
  1% 8/1014 [00:01<03:51,  4.35it/s]
  1% 9/1014 [00:01<04:06,  4.09it/s]
  1% 10/1014 [00:02<04:17,  3.90it/s]
  1% 11/1014 [00:02<04:11,  3.99it/s]
  1% 1

In [ ]:
!python scripts/video_swap.py \
-s "data/" \
-t "data/" \
-o "video_simswap_output/" \
-d 5 \
-l 1


/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Loading config: ./src/dot/simswap/configs/config.yaml
/usr/local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Total source images:  15
Total target videos:  1
Total number of face-swaps:  1
  0% 0/1 [00:00<?, ?it/s]OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Source: data/T